In [1]:
import pandas as pd
import re
from ipywidgets import interact, interactive, fixed, VBox, HBox, widgets
from IPython.display import display, HTML

# 파일 경로 (사용자님 경로로 설정)
file_path = "C:\\Users\\셜록\\Downloads\\TB_RECIPE_SEARCH_241226.csv"

# CSV 파일 불러오기
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    # 'utf-8'로 실패하면 'cp949' (혹은 'euc-kr') 시도
    df = pd.read_csv(file_path, encoding='cp949')

# 필요한 열의 결측값 처리 (문자열로 대체)
df['CKG_MTRL_CN'] = df['CKG_MTRL_CN'].fillna('')
df['CKG_STA_ACTO_NM'] = df['CKG_STA_ACTO_NM'].fillna('선택 안함')
df['CKG_KND_ACTO_NM'] = df['CKG_KND_ACTO_NM'].fillna('선택 안함')

# 레시피 데이터 확인
print(f"총 레시피 수: {len(df)}")

총 레시피 수: 23192


In [2]:
# --- 🎨 UI 위젯 설정 (기존과 동일) ---
ckg_sta_options = ['선택 안함'] + sorted(df['CKG_STA_ACTO_NM'].unique().tolist())
ckg_knd_options = ['선택 안함'] + sorted(df['CKG_KND_ACTO_NM'].unique().tolist())

material_input = widgets.Text(
    value='',
    placeholder='예: 양파, 당근, 계란 (쉼표로 구분)',
    description='필수 재료:',
    style={'description_width': 'initial', 'font_size': '16px'},
    layout=widgets.Layout(width='65%')
)

ckg_sta_select = widgets.Dropdown(
    options=ckg_sta_options,
    value='선택 안함',
    description='요리 상황:',
    style={'description_width': 'initial', 'font_size': '14px'},
    layout=widgets.Layout(width='30%')
)

ckg_knd_select = widgets.Dropdown(
    options=ckg_knd_options,
    value='선택 안함',
    description='요리 종류:',
    style={'description_width': 'initial', 'font_size': '14px'},
    layout=widgets.Layout(width='30%')
)

search_button = widgets.Button(
    description='✨ 레시피 찾아보기 ✨',
    button_style='info', # 모던한 느낌으로 'info' 스타일 사용
    tooltip='입력한 재료와 조건으로 레시피를 검색합니다.',
    layout=widgets.Layout(width='30%', height='45px', font_weight='bold')
)

output_area = widgets.Output()

# --- 🔎 검색 로직 함수 (재료 목록 및 링크 기능 포함) ---
def search_recipes(materials_str, ckg_sta, ckg_knd):
    input_materials = [m.strip().replace(' ', '') for m in re.split(r'[,| ]', materials_str) if m.strip()]
    
    if not input_materials:
        with output_area:
            output_area.clear_output()
            display(HTML("<h4 style='color: #e74c3c;'>⚠️ 재료를 입력해 주세요!</h4>"))
        return

    filtered_df = df.copy()
    if ckg_sta != '선택 안함':
        filtered_df = filtered_df[filtered_df['CKG_STA_ACTO_NM'] == ckg_sta]
    if ckg_knd != '선택 안함':
        filtered_df = filtered_df[filtered_df['CKG_KND_ACTO_NM'] == ckg_knd]
        
    if filtered_df.empty:
        with output_area:
            output_area.clear_output()
            display(HTML(f"<h4 style='color: #e74c3c;'>😭 선택한 조건에 맞는 레시피가 없습니다.</h4>"))
        return

    def get_matches_and_count(recipe_materials_str):
        if pd.isna(recipe_materials_str) or not recipe_materials_str:
            return 0, []
        
        recipe_materials_list = recipe_materials_str.split('|')
        match_count = 0
        matched_materials = []
        
        for input_mat in input_materials:
            if any(input_mat in mat.replace(' ', '') for mat in recipe_materials_list):
                match_count += 1
                matched_materials.append(input_mat)
                
        return match_count, matched_materials

    match_results = filtered_df['CKG_MTRL_CN'].apply(lambda x: get_matches_and_count(x))
    filtered_df['Match_Count'] = match_results.apply(lambda x: x[0])
    filtered_df['Matched_Materials'] = match_results.apply(lambda x: x[1])

    top_recipes = filtered_df.sort_values(by=['Match_Count', 'INQ_CNT'], ascending=[False, False])
    top_recipes = top_recipes[top_recipes['Match_Count'] > 0]
    
    # RCP_SNO, Matched_Materials 포함
    final_results = top_recipes[['CKG_NM', 'CKG_TIME_NM', 'Match_Count', 'RCP_SNO', 'Matched_Materials']].head(7)
    
    # 5. 결과 출력
    with output_area:
        output_area.clear_output()
        
        if final_results.empty:
            display(HTML(f"<h4 style='color: #e74c3c;'>입력한 재료를 포함하는 레시피를 찾지 못했습니다.</h4>"))
            return

        # 모던하고 예쁜 스타일 적용
        result_title = f"""
        <h3 style='color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px; margin-top: 20px;'>
            ✅ 추천 레시피 ({len(final_results)}개)
        </h3>
        <p style='font-size: 14px; color: #7f8c8d;'>
            입력 재료: <b>{', '.join(input_materials)}</b>
        </p>
        """
        display(HTML(result_title))

        # 결과를 보기 좋게 HTML 표로 변환
        html_table = f"""
        <style>
            .recipe-table {{
                width: 100%; border-collapse: collapse; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1); border-radius: 8px; overflow: hidden;
            }}
            .recipe-table th, .recipe-table td {{
                border: 1px solid #ecf0f1; padding: 12px; text-align: left;
            }}
            .recipe-table thead tr {{
                background-color: #3498db; color: white;
            }}
            .recipe-table tbody tr:nth-child(even) {{
                background-color: #f9f9f9;
            }}
            .recipe-table tbody tr:hover {{
                background-color: #eaf2f8; transition: background-color 0.3s;
            }}
            .recipe-link {{
                color: #2980b9; text-decoration: none; font-weight: bold;
            }}
            .recipe-link:hover {{
                text-decoration: underline; color: #1e6293;
            }}
            .match-count {{
                color: #27ae60; font-weight: bold; text-align: center;
            }}
        </style>
        <table class='recipe-table'>
            <thead>
                <tr>
                    <th style='width: 35%;'>레시피 제목 (클릭)</th>
                    <th style='width: 15%; text-align: center;'>소요 시간</th>
                    <th style='width: 15%; text-align: center;'>재료 일치 수</th>
                    <th style='width: 35%;'>일치된 재료</th>
                </tr>
            </thead>
            <tbody>
        """
        for index, row in final_results.iterrows():
            recipe_link = f"<a href='https://www.10000recipe.com/recipe/{row['RCP_SNO']}' target='_blank' class='recipe-link'>{row['CKG_NM']}</a>"
            matched_materials_str = ', '.join(row['Matched_Materials'])
            
            html_table += f"""
            <tr>
                <td>{recipe_link}</td>
                <td style='text-align: center;'>{row['CKG_TIME_NM']}</td>
                <td class='match-count'>{row['Match_Count']}개</td>
                <td>{matched_materials_str}</td>
            </tr>
            """
        html_table += "</tbody></table>"
        
        display(HTML(html_table))


# --- 🔗 이벤트 연결 (기존과 동일) ---
def on_button_click(b):
    search_recipes(material_input.value, ckg_sta_select.value, ckg_knd_select.value)

search_button.on_click(on_button_click)


# --- 🖥️ 메인 UI 레이아웃 구성 ---
# 메인 화면 레이아웃 (VBox 위젯)
input_and_button = HBox([
    material_input,
    search_button
], layout=widgets.Layout(align_items='flex-end', width='100%', justify_content='space-between'))

condition_section = HBox([
    ckg_sta_select, 
    widgets.Label(value="", layout=widgets.Layout(width='5%')), # 간격 조절
    ckg_knd_select
], layout=widgets.Layout(justify_content='center', width='100%'))

main_ui_elements = [
    widgets.HTML("<h2>🍳 레시피 추천 시스템</h2><p>✨ **가지고 계신 재료**를 입력하고, 요리 조건(선택 사항)을 선택하여 레시피를 추천받으세요!</p>"),
    input_and_button,
    condition_section,
    widgets.HTML("<hr style='border: 1px solid #bdc3c7;'>"),
    output_area
]

main_ui = VBox(main_ui_elements, layout=widgets.Layout(padding='20px', border='1px solid #ecf0f1', border_radius='10px', background='white'))

In [5]:
from ipywidgets import VBox, widgets
from IPython.display import display, HTML, Javascript

# --- 🖥️ 시작 화면 (Splash Screen) 정의 ---
# (이 부분의 HTML/CSS는 변경 없이 유지)
splash_html = """
<style>
    /* 전역 스타일: 메인 화면 위젯에도 적용될 수 있도록 일부 스타일 추가 */
    body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background-color: #f4f7f6; }
    
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(30px); }
        to { opacity: 1; transform: translateY(0); }
    }
    .splash-container {
        text-align: center; 
        padding: 80px 40px; 
        /* 배경: 부드러운 파스텔톤 그라디언트 */
        background: linear-gradient(135deg, #f0f7ff 0%, #e0e6f0 100%); 
        border-radius: 25px; 
        /* 그림자: 부유하는 듯한 깊은 그림자 */
        box-shadow: 0 15px 30px rgba(44, 62, 80, 0.2); 
        width: 90%; 
        max-width: 650px; 
        margin: 80px auto; 
        border: 1px solid #dcdcdc;
    }
    .splash-title {
        font-size: 3.5em; 
        color: #e67e22; /* 오렌지 계열 포인트 컬러 */
        margin-bottom: 10px;
        text-shadow: 3px 3px 6px rgba(0, 0, 0, 0.15);
        font-weight: 800; /* 더 굵게 */
        animation: fadeIn 1s ease-out;
    }
    .splash-subtitle {
        font-size: 1.4em; 
        color: #34495e; /* 다크 블루 */
        margin-bottom: 50px;
        font-weight: 300;
        animation: fadeIn 1.5s ease-out;
    }
    .start-button {
        /* 버튼 스타일: 플랫 디자인 & 생동감 있는 색상 */
        background-color: #2ecc71; 
        color: white; 
        border: none; 
        padding: 18px 40px; 
        font-size: 20px; 
        cursor: pointer; 
        border-radius: 10px; 
        transition: all 0.3s;
        box-shadow: 0 5px #27ae60; /* 3D 효과 */
        font-weight: bold;
        letter-spacing: 1px;
    }
    .start-button:hover {
        background-color: #27ae60;
        transform: translateY(2px);
        box-shadow: 0 3px #1e8449;
    }

    /* 메인 화면 스타일링 (위젯 오버라이드) */
    #main_app_container .widget-vbox, #main_app_container .widget-hbox {
        background-color: #ffffff; /* 메인 배경을 흰색으로 통일 */
    }
    #main_app_container h2 {
        color: #2c3e50; /* 다크 타이틀 */
        border-bottom: 3px solid #f1c40f; /* 노란색 강조선 */
        padding-bottom: 10px;
    }
    #main_app_container p {
        color: #7f8c8d;
    }
</style>
<div class="splash-container">
    <div class="splash-title">지운이의 레시피 추천!!!!!!!</div>
    <div class="splash-subtitle">🌟 나만의 맞춤 레시피를 지금 바로 찾아보세요! 👩‍🍳</div>
    <button class="start-button" onclick="startApp()">시작하기</button>
</div>
"""

# HTML을 출력할 위젯
splash_output = widgets.Output()
with splash_output:
    display(HTML(splash_html))


# --- ⚙️ 화면 전환 로직 (Javascript) ---

# 메인 UI를 HTML/JavaScript 영역에 렌더링하기 위해 HTML로 변환
# 주의: Jupyter 위젯의 HTML 표현에는 중괄호가 포함되어 오류 발생 가능
main_ui_html = main_ui._repr_html_()

# 시작 버튼 클릭 시 실행될 JavaScript 함수
# Python F-string 대신 .format()을 사용하여 중괄호 충돌 방지
js_code_template = """
<script>
    function startApp() {{
        // 스플래시 화면이 있는 DOM 요소 숨기기
        const splash = document.getElementById('splash_screen_container');
        if (splash) {{
            splash.style.display = 'none';
        }}
        
        // 메인 화면을 출력할 DOM 요소에 메인 UI HTML 삽입
        const main_container = document.getElementById('main_app_container');
        if (main_container) {{
            // 메인 UI HTML을 JavaScript 템플릿 리터럴 안에 삽입
            main_container.innerHTML = `{main_ui_html_placeholder}`; 
        }}
    }}
</script>
"""

# HTML 변수를 JavaScript 문자열로 안전하게 삽입
# 1. 백틱을 이스케이프하고
safe_html = main_ui_html.replace('`', '\\`')
# 2. .format()을 사용하여 문자열을 완성합니다.
js_code = js_code_template.replace('{main_ui_html_placeholder}', safe_html)


# 전체 UI 구성 (시작 화면과 메인 화면 컨테이너)
full_app_layout = VBox([
    widgets.HTML(f"<div id='splash_screen_container'>{splash_output._repr_html_()}</div>"),
    widgets.HTML(f"<div id='main_app_container'></div>"),
    widgets.HTML(js_code) # JavaScript 코드 삽입
])

# 최종 UI 출력
display(full_app_layout)

AttributeError: 'VBox' object has no attribute '_repr_html_'